In [1]:
import carla, time, cv2
import numpy as np
from Utility import spawn_camera, speed_control, keep_lane, control_retro, find_point
from ParkingIdentifier import process_image

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [8]:
world.set_weather(carla.WeatherParameters.WetNight)

In [3]:
spawn_point = carla.Transform(carla.Location(4, -30, 2))
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
vehicleO = world.spawn_actor(vehicle_bp, spawn_point)

In [ ]:
vehicleO.destroy()

In [3]:
spawn_point = carla.Transform(carla.Location(-2.5, -23, 2), carla.Rotation(yaw=100))

In [4]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-0.8, y=0.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True
video_output=None 
center = None
firstTime = True

In [5]:
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range
radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=vehicle)

In [6]:
def camera_rigth_callback(image):
    global run, video_output, center
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    prediction, c=process_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    if prediction and run and c[0] > -100 and c[0] < 100:
        cv2.imwrite("prova.png", np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
        center = c
        print(center)
        run = False
camera_rigth.listen(lambda image: camera_rigth_callback(image))

min_ttc = float('inf')
def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')
    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            if ttc < min_ttc:
                min_ttc = ttc

radar.listen(radar_callback)

In [ ]:
target_speed_mps = 8 / 3.6

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0
target_distance = 3.3
distance_travelled = 0.0

n_listem_break=0
ttc_threshold = 1.0
collision = False
time.sleep(1.5)
while run:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    control, angolo = keep_lane(video_output, control, current_speed_mps, target_speed_mps)
    vehicle.apply_control(control)
    if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  
            vehicle.apply_control(control)
            print("Emergency braking activated!")
            n_listem_break += 1
            if n_listem_break>10:
                collision = True
                break
    time.sleep(0.05)
target_distance += -(center[0]/280)
print(target_distance)
pre_time = time.time()
while distance_travelled < target_distance and not collision:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * (time.time() - pre_time)
    pre_time = time.time()
    control, _ = keep_lane(video_output, control, current_speed_mps, target_speed_mps)
    vehicle.apply_control(control)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
control.brake = 0.0
control.reverse = True
control.steer = 0.8
distance_travelled = 0.0
target_distance = 3.9
pre_time = time.time()
while distance_travelled < target_distance and not collision:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * (time.time() - pre_time)
    pre_time = time.time()
    control = speed_control(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
camera_rigth.destroy()
image1 = None
def camera_callback2(image):
    global image1
    image1 =  np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
camera_retro.listen(lambda image: camera_callback2(image))
time.sleep(0.15)
while True:
    if control_retro(vehicle, find_point(image1), control, target_speed_mps=6/3.6):
        break
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
vehicle.destroy()

95.24999938093168
95.24999938093168
96.55555532065134
96.55555532065134
96.87499956844704
96.87499956844704
95.1428568867103
95.1428568867103
96.87499956844704
96.87499956844704
87.29999963908674
87.29999963908674
87.49999987232522
87.49999987232522
95.99999957255706
95.99999957255706
95.49999967247977
95.49999967247977
95.1428568867103
95.1428568867103
95.81818155125967
95.81818155125967
95.1428568867103
95.1428568867103
86.55555409880387
86.55555409880387
95.29999875622238
95.29999875622238
95.03030246548892
95.03030246548892
85.24999929744908
85.24999929744908
86.26315693578839
86.26315693578839
87.66666559499242
87.66666559499242
86.09677381751304
86.09677381751304
95.37209246715065
95.37209246715065
87.31249935484337
87.31249935484337
86.73333299459664
86.73333299459664
87.04761848995517
87.04761848995517
86.5714284341763
86.5714284341763
84.73333293894156
84.73333293894156
86.68181728626386
86.68181728626386
86.06249939120676
86.06249939120676
86.33333247120143
86.33333247120143


True

In [25]:
vehicle.destroy()

True